In [1]:
from beat_tracking.models import my_madmom, RNNJointBeatProcessor
from beat_tracking.data_loading import get_midi_filelist, get_pianoroll_dataset, get_dataloader
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import pretty_midi as pm
import mir_eval
import numpy as np

AMAPS, AMAPS_path_dict = get_midi_filelist(["AMAPS"])
CPM, CPM_path_dict = get_midi_filelist(["CPM"])
ASAP, ASAP_path_dict = get_midi_filelist(["ASAP"])

df2 = pd.read_csv("metadata.csv")
all_datasets, all_datasets_path_dict = get_midi_filelist(["ASAP","CPM","AMAPS"])

/home/thomass/miniconda3/envs/env_02/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Dataloaders:
train_loader_asap = get_dataloader(get_pianoroll_dataset(ASAP,ASAP_path_dict),"train",shuffle=True)
train_loader_amaps = get_dataloader(get_pianoroll_dataset(AMAPS,AMAPS_path_dict),"train",shuffle=True)
train_loader_cpm = get_dataloader(get_pianoroll_dataset(CPM,CPM_path_dict),"train",shuffle=True)
val_loader_asap = get_dataloader(get_pianoroll_dataset(ASAP,ASAP_path_dict),"val",shuffle=False)
val_loader_amaps = get_dataloader(get_pianoroll_dataset(AMAPS,AMAPS_path_dict),"val",shuffle=False)
val_loader_cpm = get_dataloader(get_pianoroll_dataset(CPM,CPM_path_dict),"val",shuffle=False)
test_loader_asap = get_dataloader(get_pianoroll_dataset(ASAP,ASAP_path_dict),"test",shuffle=False)
test_loader_amaps = get_dataloader(get_pianoroll_dataset(AMAPS,AMAPS_path_dict),"test",shuffle=False)
test_loader_cpm = get_dataloader(get_pianoroll_dataset(CPM,CPM_path_dict),"test",shuffle=False)

print("Lengths of training loaders:",len(train_loader_asap),len(train_loader_amaps),len(train_loader_cpm))
print("Lengths of validation loaders:",len(val_loader_asap),len(val_loader_amaps),len(val_loader_cpm))
print("Lengths of test loaders:",len(test_loader_asap),len(test_loader_amaps),len(test_loader_cpm))

Lengths of training loaders: 932 169 268
Lengths of validation loaders: 105 15 40
Lengths of test loaders: 0 29 0


In [3]:
def evaluate(dataset,split):
    processor = RNNJointBeatProcessor()
    name = "scores_" + str(dataset)
    scores = []
    strr = split+"_loader_"+dataset
    for i,element in enumerate(globals()[strr]):
        beats_pred = processor.process(element[4][0])
        #if dataset == "asap":
        #    annotation_file = element[4][0][:-4]+"_annotations.txt"
        #    beats_targ = beat_list_to_array(annotation_file,"annotations","beats")
        #else:
        #    midi_data = pm.PrettyMIDI(element[4][0])
        #    beats_targ = midi_data.get_beats()
        beats_targ = element[1]
        beats_pred_trimmed = mir_eval.beat.trim_beats(beats_pred)
        beats_targ_trimmed = mir_eval.beat.trim_beats(beats_targ)
        f1 = mir_eval.beat.f_measure(beats_targ_trimmed, beats_pred_trimmed)
        scores.append(f1)
        if i%10 == 0:
            print(i,element[4][0],f1)
    return scores

In [4]:
amaps_val_scores = evaluate("amaps","val")
print("Summary:",np.sum(amaps_val_scores)/len(amaps_val_scores))

0 data/A-MAPS_1.2/MAPS_MUS-alb_se4_AkPnBsdf.mid 0.983751846381093
10 data/A-MAPS_1.2/MAPS_MUS-scn16_5_SptkBGAm.mid 0.9625984251968505
Summary: 0.9540315488367525


In [5]:
cpm_val_scores = evaluate("cpm","val")
print("Summary:",np.sum(cpm_val_scores)/len(cpm_val_scores))

0 data/Piano-MIDI/midis/Albeniz/alb_se4.mid 0.970674486803519
10 data/Piano-MIDI/midis/Chopin/chpn-p22.mid 0.9444444444444444
20 data/Piano-MIDI/midis/Haydn/hay_40_2.mid 0.9771986970684038
30 data/Piano-MIDI/midis/Rachmaninoff/rac_op23_5.mid 0.9878048780487805
Summary: 0.947501809948422


In [6]:
asap_val_scores = evaluate("asap","val")
print("Summary:",np.sum(asap_val_scores)/len(asap_val_scores))

0 data/asap-dataset/Bach/Fugue/bwv_858/VuV01M.mid 0.9657794676806084
10 data/asap-dataset/Bach/Prelude/bwv_856/LuoJ01M.mid 0.8085106382978724
20 data/asap-dataset/Beethoven/Piano_Sonatas/16-1/Woo01.mid 0.8963855421686747
30 data/asap-dataset/Beethoven/Piano_Sonatas/21-1_no_repeat/LeeN02M.mid 0.8443548387096773
40 data/asap-dataset/Beethoven/Piano_Sonatas/21-1_no_repeat/Sychev01.mid 0.8302180685358256
50 data/asap-dataset/Beethoven/Piano_Sonatas/26-1/HuNY02M.mid 0.8783204798628963
60 data/asap-dataset/Beethoven/Piano_Sonatas/29-4/ChowK05M.mid 0.7755261106780982
70 data/asap-dataset/Chopin/Etudes_op_10/12/Mordvinov04.mid 0.6878612716763006
80 data/asap-dataset/Chopin/Etudes_op_25/12/KWON03.mid 0.7
90 data/asap-dataset/Schubert/Impromptu_op.90_D.899/3/Hou06M.mid 0.8869047619047619
100 data/asap-dataset/Schubert/Impromptu_op.90_D.899/3/ZhangW07M.mid 0.7439198855507868
Summary: 0.8393444068058197
